In [42]:
import subprocess
from astropy.table import Table
import os
import csv
#in the furture make packages and import common functions
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pyraf import gwm
from astropy.io import ascii
import numpy as np
from astropy.stats import sigma_clip
from astropy.io import fits
from astropy.io import ascii
import subprocess

In [17]:
from pyraf import iraf
from iraf import obsutil
from iraf import psfmeasure
from iraf import phot
from iraf import daofind
from iraf import daophot
from iraf import psf
from iraf import artdata
from iraf import mkobjects
from iraf import starlist

In [38]:
imagename = '/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits'
diffImage = '/home/mj1e16/Simages/diff_44_1.fits'
SImage= '/home/mj1e16/Simages/testsimage.fits'
ext =  79

In [43]:
outDir = '/home/mj1e16/iraf/outputs/' # for iraf output files ONLY
imageDir = '/home/mj1e16/Simages/irafImages/' # for images generated during process ONLY
simageDir = '/home/mj1e16/Simages/' # directory for simulated images
irafDir = '/home/mj1e16/iraf/' #Directory containing iraf installation
starlistDir = '/home/mj1e16/starlists/'

AttributeError: 'module' object has no attribute 'irafOutputDir'

In [20]:
def mkobjectspluspsf(imagename,diffimage,diffPlusSim,ext,section,minMag,maxMag,Xmax=1000,Xmin=0,Ymax=1000,Ymin=1000,thresh=50000,nsegs=5,goodpsf='',spatialDist='uniform',zeropoint=25.):
    
    daoout = outDir+'{}_{}daofindone'.format(ext,section)
    photout = outDir+'{}_{}photone'.format(ext,section)
    psfout = outDir+'{}_{}psfone'.format(ext,section)
    groupout = outDir+'{}_{}groupfileone'.format(ext,section)
    opstout = outDir+'{}_{}optsone'.format(ext,section)
    starfieldsect = outDir+'starfield_{}.dat'.format(section)
    imseg = imageDir+'imageseg{}.fits'.format(section)
    
    hdu = fits.open(imagename)
    print(imagename)
    image = hdu[ext].section[Xmin:Xmax,Ymin:Ymax]
    hdu = fits.PrimaryHDU(image)
    hdu.writeto(imseg,clobber=True)
    
    with open(starfieldsect,'w') as f:
        f.write("")
    
    daofind(imseg,output=daoout,sigma=1.0,threshold=thresh)
    phot(imseg,skyfile=daoout,coords=daoout,output=photout,interactive='no')
    psf(imseg,photfile=daoout,pstfile=photout,psfimage=psfout,opstfile=opstout,groupfile=groupout,interactive='no')
    starlist(starfieldsect,nstars=1000/nsegs,xmin=Xmin,xmax=Xmax,ymin=Ymin,ymax=Ymax,spatial=spatialDist,minmag=minMag,maxmag=maxMag)
    subprocess.call(['cp',starfieldsect,'/home/mj1e16/starlists/starfield_{}_{}.dat'.format(section,minMag)])
    try:
        mkobjects(diffimage,output=diffPlusSim,objects=starfieldsect,gain=110,rdnoise=127,star=psfout+'.fits')
        
    except:
        try:
            #subprocess.call(['rm','/home/mj1e16/Simages/testsimage*'])
            mkobjects(diffimage,output=diffPlusSim,objects=starfieldsect,gain=110,rdnoise=127,star=goodpsf)
            print('Waring: Section {} with bad PSF, {} used inplace'.format(section,goodpsf))
        except:
            print('ERROR: PSF for Section {} should be made manually'.format(section))
    
    return psfout+'.fits'

In [33]:
def findPSFandMkObjects(imageName,diffImage,SImage,extension,minMag,maxMag,imageshape=[1015., 1085.],pixelsize=3.98):
    
    print(imageName)
    
    os.chdir(irafDir)
    
    xlength = imageshape[1]
    ylength = imageshape[0]
    nsegs = 4
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    PSF = []
    goodpsf=''
    for xsegs in range(nsegs):
        lowx = int(xsegs*xsegment)+15
        highx = int(lowx+xsegment)+15-10
        psf = []
        for ysegs in range(0,nsegs):
            lowy = int(ysegs*ysegment)+15
            highy = int(lowy+ysegment)+15-10
            
            
            # stars are being lost from images in xseg=0
            SImage= imageDir+'testsimage_{}_{}_{}.fits'.format(xsegs,ysegs,minMag)
            
            goodpsf = mkobjectspluspsf(imageName,diffImage,SImage,extension,'{}_{}'.format(xsegs,ysegs),minMag,maxMag,Xmax=highx,Xmin=lowx,Ymax=highy,Ymin=lowy,nsegs=nsegs**2,goodpsf=goodpsf)
            
            b = diffImage.find('{}_alt'.format(minMag))
            if b == -1:
                diffImage = diffImage[:-5]+'{}_alt.fits'.format(minMag)
            try:
                hdu = fits.open(SImage) #,ignore_missing_end=True)
                imdata = hdu[0].data
                hdu = fits.PrimaryHDU(imdata)
                hdu.writeto(diffImage,clobber=True)
                print('find me')
                print(SImage)
                print(diffImage)
            except:
                print('bad at section {} {}'.format(xsegs,ysegs))

            

            
    return 'complete!'

In [22]:
def cleanDirectories(dirlist):
    # clean iraf output and input images
    # make sure to have no needed files in these directories
    for directory in dirlist:
        dirfiles = os.listdir(directory)
        for eachFile in dirfiles:
            #print('rm',directory+eachFile)
            subprocess.call(['rm',directory+eachFile])

In [23]:
def stitchStarList(nsegs,mag,starDir='/home/mj1e16/starlists/'):
    with open(starDir+'starfield_0_0_{}.dat'.format(mag)) as f:
        lines = f.readlines()
    bigstring = ['# X_POS Y_POS MAG\n']
    bigstring.extend(lines[0:18])
    for x in range(nsegs):
        for y in range(nsegs):
            with open(starDir+'starfield_{}_{}_{}.dat'.format(x,y,mag)) as f:
                lines = f.readlines()
            bigstring.extend(lines[18:])


    with open('/home/mj1e16/keplerPhotometry/starlistFull_{}.dat'.format(mag),'w') as f:
        f.write(''.join(bigstring))


In [34]:
cleanDirectories([outDir,imageDir]) # cleans iraf output files, will remove all files from these directories

In [39]:
run = 'not complete'
ext =  44
medianNum = 1
diffImage = '/home/mj1e16/Simages/diff_{}_{}.fits'.format(ext,medianNum)
for mag in range(-7,0):
    cleanDirectories([outDir,imageDir]) # cleans iraf output files, will remove all files from these directories
    for x in range(5):
        try:
            run = findPSFandMkObjects(imagename,diffImage,SImage,ext,mag,(mag+1))
            if run == 'complete!':
                break
        except:
            pass
    dirlist = os.listdir('/home/mj1e16/starlists/')
    dirlist = [x for x in dirlist if '.dat' in x]
    for x in dirlist:
        subprocess.call(['mv',starlistDir+x,starlistDir+'{}/{}/'.format(ext,medianNum)])

/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg0_0.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

   65.46    45.84   -1.239   0.719  -0.291  -0.154     1
  196.76    64.16   -0.335   0.994  -0.143  -0.138     2
  182.31   109.30   -0.749   0.866   0.021  -0.123     3
   23.49   124.91   -1.297   0.631  -0.121  -0.050     4
  188.01   146.52   -1.251   0.754   0.237   0.382 

imageseg0_1.fits     61.90    76.99  178.4813   10.006  ok
imageseg0_1.fits     24.44   118.47  172.6976   11.620  ok
imageseg0_1.fits     92.61   130.72  175.1868   10.227  ok
imageseg0_1.fits     67.56   152.35  173.0547   11.705  ok
imageseg0_1.fits    190.28   163.60  174.1605   11.679  ok
imageseg0_1.fits    232.52   205.37  180.4563    9.608  ok
imageseg0_1.fits     68.00   222.49  174.4027   10.625  ok
imageseg0_1.fits     45.52   238.92  173.6942   11.186  ok
imageseg0_1.fits    227.73   250.74  174.4844   10.874  ok
imageseg0_1.fits    250.71   272.45  173.7453   11.666  ok

Analytic psf function(s) (gauss): 	Analytic psf function(s): gauss
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (11.): 	New psf radius: 11. scale units 11. pixels
Fitting radius in scale units (3.): 	New fitting radius: 3. scale units 3. pixels
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or v

Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg0_3.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

   99.73    15.52   -1.176   0.428  -0.065   0.527     1
   34.96    38.38   -1.061   0.849   0.263   0.324     2
  109.41    40.97   -1.171   0.649   0.023  -0.106     3
  207.02    70.65   -0.015   0.960   0.090   0.124     4
   34.64    84.06   -1.086   0.511   0.128   0.239     5
  221.04    94.40   -1.159   0.668   0.461   0.683     6
   76.07   118.42   -1.116   0.507   0.255   0.497     7
   43.82   120.34   -0.649   0.996  -0.137   0.078     8
   88.53   122.80   -0.030   0.955   0

Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in scale units (3.) (CR or value): 	Aperture radius 1: 3. scale units 3. pixels
Standard deviation of background in counts (0.) (CR or value): 	New standard deviation of background: 0. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts

imageseg1_0.fits     22.71     9.58  176.2863   11.184  ok
imageseg1_0.fits    114.87    25.62   179.214   10.973  ok
imageseg1_0.fits     92.60    67.83  171.3671   11.485  ok
imageseg1_0.fits    178.07    72.91  175.6666   10.184  ok
imageseg1_0.fits     20.90    87.79  177.2997   10.558  ok
imageseg1_0.fits    249.98    91.74  178.6948   10.915  ok
imageseg1_0.fits    232.54    97.90  178.6481   11.466  ok
imageseg1_0.fits     27.48   101.58  177.3387   11.158  ok
imageseg

	X:   55.84 Y:   88.26  Mag:  -1.380  Dmin: 150.0795  Dmax: 176387.4
Star 5 has been added to the PSF star list
	X:   90.88 Y:   90.28  Mag:  -1.271  Dmin: 121.9487  Dmax: 162052.4
Star 6 has been added to the PSF star list
	X:   17.70 Y:   90.50  Mag:  -1.026  Dmin: 87.62771  Dmax: 159044.3
Star 7 has been added to the PSF star list
	X:  159.53 Y:  121.88  Mag:  -0.511  Dmin: 167.8291  Dmax: 101037.9
Star 8 has been added to the PSF star list
	X:  194.68 Y:  155.84  Mag:  -1.150  Dmin: 162.9639  Dmax: 151467.5
Star 9 has been added to the PSF star list
	X:   54.80 Y:  167.29  Mag:  -1.181  Dmin: 100.7867  Dmax: 161001.1
Star 10 has been added to the PSF star list
	X:  138.16 Y:  172.49  Mag:  -0.593  Dmin: 172.787  Dmax: 99905.87
Star 11 has been added to the PSF star list
	X:   38.40 Y:  204.58  Mag:  -1.298  Dmin: 172.5997  Dmax: 177395.

11 PSF stars read from /home/mj1e16/iraf/outputs/63_1_1photone


Fitting function gauss    norm scatter: 0.04888337

Analytic PSF fit 
    Functio

...

Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts

imageseg2_1.fits    243.43    11.46  187.5807   10.568  ok
imageseg2_1.fits    224.83    33.01  189.5688    9.523  ok
imageseg2_1.fits     42.81    39.85  179.8485   10.792  ok
imageseg2_1.fits    233.58    61.41  184.8785   11.339  ok
imageseg2_1.fits    237.59   105.18  180.3151   11.175  ok
imageseg2_1.fits    151.04   142.84  180.2576   11.020  ok
imageseg2_1.fits    154.80   188.47   179.459   10.911  ok
imageseg2_1.fits    126.98   205.64  179.5064   11.526  ok
imageseg2_1.fits    184.04   219.38  178.0009   11.181  ok
imageseg2_1.fits    173.54   224.30  179.7674   11.126  ok

Analytic psf function(s) (gauss): 	Analytic psf function(s): gauss
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (11.): 	New psf radius: 11. scale units 11. pixels
Fitting radius in scale units (3.): 	New fitting radius: 3. scale units 3. pixels
Minimum good data value (INDEF) (CR or 


Analytic PSF fit 
    Function: gauss  X: 128.5  Y: 137.5  Height: 156113.8  Psfmag: 10.565
    Par1: 0.7787859  Par2: 1.06989

Computed 1 lookup table(s)

Writing PSF image /home/mj1e16/iraf/outputs/63_2_2psfone.fits
Writing output PSF star list /home/mj1e16/iraf/outputs/63_2_2optsone
Writing output PSF star group file /home/mj1e16/iraf/outputs/63_2_2groupfileone
find me
/home/mj1e16/Simages/irafImages/testsimage_2_2_-1.fits
/home/mj1e16/Simages/diff_63_1-1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum

Star 25 has been added to the PSF star list
	X:  168.53 Y:  267.14  Mag:  -0.210  Dmin: 181.6756  Dmax: 135452.5

25 PSF stars read from /home/mj1e16/iraf/outputs/63_2_3photone


Fitting function gauss    norm scatter: 0.0972808

Analytic PSF fit 
    Function: gauss  X: 128.5  Y: 137.5  Height: 145241.7  Psfmag: 10.504
    Par1: 0.7096537  Par2: 0.9432076

Computed 1 lookup table(s)

Writing PSF image /home/mj1e16/iraf/outputs/63_2_3psfone.fits
Writing output PSF star list /home/mj1e16/iraf/outputs/63_2_3optsone
Writing output PSF star group file /home/mj1e16/iraf/outputs/63_2_3groupfileone
find me
/home/mj1e16/Simages/irafImages/testsimage_2_3_-1.fits
/home/mj1e16/Simages/diff_63_1-1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in s

Standard deviation of background in counts (0.) (CR or value): 	New standard deviation of background: 0. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts

imageseg3_1.fits     12.01    35.11  178.0316   10.005  ok
imageseg3_1.fits     74.32    56.02  172.9415   11.936  ok
imageseg3_1.fits     16.01    63.38  176.0032   11.476  ok
imageseg3_1.fits      5.37    63.93  175.6358   11.222  ok
imageseg3_1.fits     79.74    76.33  176.0816   10.991  ok
imageseg3_1.fits     47.67   102.87  180.6225    9.373  ok
imageseg3_1.fits    230.74   127.09  197.3455    9.408  ok
imageseg3_1.fits    233.24   145.28  190.1813   10.344  ok
imageseg3_1.fits     93.17   170.29  176.1976   11.831  ok
imageseg3_1.fits     93.67   182.57  177.3964    9.876  ok

Analytic psf function(s) (gauss): 	Analytic psf function(s): gauss
Order of variable psf (0): 	Order of variable psf

    Function: gauss  X: 128.5  Y: 120.5  Height: 92716.08  Psfmag: 11.442
    Par1: 0.8422139  Par2: 2.250882

Computed 1 lookup table(s)

Writing PSF image /home/mj1e16/iraf/outputs/63_3_2psfone.fits
Writing output PSF star list /home/mj1e16/iraf/outputs/63_3_2optsone
Writing output PSF star group file /home/mj1e16/iraf/outputs/63_3_2groupfileone
find me
/home/mj1e16/Simages/irafImages/testsimage_3_2_-1.fits
/home/mj1e16/Simages/diff_63_1-1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: 

In [62]:
mag = -7
for x in range(5):
        try:
            run = findPSFandMkObjects(imagename,diffImage,SImage,ext,mag,(mag+1))
            if run == 'complete!':
                break
        except:
            pass

/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg0_0.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

   65.46    45.84   -1.239   0.719  -0.291  -0.154     1
  196.76    64.16   -0.335   0.994  -0.143  -0.138     2
  182.31   109.30   -0.749   0.866   0.021  -0.123     3
   23.49   124.91   -1.297   0.631  -0.121  -0.050     4
  188.01   146.52   -1.251   0.754   0.237   0.382 

imageseg0_1.fits     55.35    38.66  177.4142   10.965  ok
imageseg0_1.fits     95.25    45.96    181.78   10.939  ok
imageseg0_1.fits     62.99    55.72   181.623   11.109  ok
imageseg0_1.fits    198.80    64.12  177.4327   11.102  ok
imageseg0_1.fits     61.90    76.99  178.4813   10.006  ok
imageseg0_1.fits     24.44   118.47  172.6976   11.620  ok
imageseg0_1.fits     92.61   130.72  175.1868   10.227  ok
imageseg0_1.fits    260.75   141.69  175.7717   11.639  ok
imageseg0_1.fits     67.56   152.35  173.0547   11.705  ok
imageseg0_1.fits    190.28   163.60  174.1605   11.679  ok
imageseg0_1.fits    232.52   205.37  180.4563    9.608  ok
imageseg0_1.fits     68.00   222.49  174.4027   10.625  ok
imageseg0_1.fits     45.52   238.92  173.6942   11.186  ok
imageseg0_1.fits    227.73   250.74  174.4844   10.874  ok
imageseg0_1.fits    250.71   272.45  173.5905   11.666  ok

Analytic psf function(s) (gauss): 	Analytic psf function(s): gauss
Order of variable psf (0): 	Order of variable p


9 PSF stars read from /home/mj1e16/iraf/outputs/63_0_2photone


Fitting function gauss    norm scatter: 0.08039676

Analytic PSF fit 
    Function: gauss  X: 133.5  Y: 142.5  Height: 158858.8  Psfmag: 10.242
    Par1: 0.662334  Par2: 0.9577547

Computed 1 lookup table(s)

Writing PSF image /home/mj1e16/iraf/outputs/63_0_2psfone.fits
Writing output PSF star list /home/mj1e16/iraf/outputs/63_0_2optsone
Writing output PSF star group file /home/mj1e16/iraf/outputs/63_0_2groupfileone
find me
/home/mj1e16/Simages/irafImages/testsimage_0_2_-7.fits
/home/mj1e16/Simages/diff_79_1-7_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) 

  249.98    91.74   -1.144   0.879   0.102   0.194     7
  232.53    97.90   -0.577   0.972  -0.112  -0.283     8
   27.48   101.58   -0.591   0.743  -0.610  -0.123     9
  180.89   107.39   -0.216   0.967  -0.097   0.158    10
    4.57   116.11   -0.218   0.990  -0.354  -0.257    11
  146.08   129.37   -1.253   0.733   0.456   0.309    12
  175.94   137.61   -0.147   0.870   0.032   0.312    13
  186.13   167.21   -1.142   0.901   0.045   0.057    14
    9.27   185.13   -0.277   0.989  -0.104  -0.184    15
  128.92   216.83   -1.398   0.557   0.316   0.588    16
  118.15   246.72   -0.001   0.976  -0.160   0.013    17

threshold: 50000. relerr: 1.140  0.2 <= sharp <= 1.  -1. <= round <= 1. 


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus i

imageseg1_1.fits    194.68   155.84  176.9819   10.745  ok
imageseg1_1.fits     54.80   167.29  176.6485   10.748  ok
imageseg1_1.fits    138.16   172.49  176.3111   11.322  ok
imageseg1_1.fits     38.40   204.58   177.848   10.250  ok
imageseg1_1.fits    243.43   282.46  186.8035   10.567  ok

Analytic psf function(s) (gauss): 	Analytic psf function(s): gauss
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (11.): 	New psf radius: 11. scale units 11. pixels
Fitting radius in scale units (3.): 	New fitting radius: 3. scale units 3. pixels
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Computing PSF for image: /home/mj1e16/Simages/irafImages/imageseg1_1.fits
12 stars read from /home/mj1e16/iraf/outputs/63_1_1daofindone

Star 1 has been added to the PSF star list
	X:  186.72 Y:   42.32  Mag:  -0.271  Dmin: 162.2628  Dmax: 815


Analytic PSF fit 
    Function: gauss  X: 133.5  Y: 142.5  Height: 150793.8  Psfmag: 10.7
    Par1: 0.754975  Par2: 1.431686

Computed 1 lookup table(s)

Writing PSF image /home/mj1e16/iraf/outputs/63_1_2psfone.fits
Writing output PSF star list /home/mj1e16/iraf/outputs/63_1_2optsone
Writing output PSF star group file /home/mj1e16/iraf/outputs/63_1_2groupfileone
find me
/home/mj1e16/Simages/irafImages/testsimage_1_2_-7.fits
/home/mj1e16/Simages/diff_79_1-7_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum g

imageseg2_0.fits     98.16   136.75  183.0102    9.345  ok
imageseg2_0.fits     58.57   175.75  178.7354   10.215  ok
imageseg2_0.fits    242.70   220.26  184.1949    9.832  ok
imageseg2_0.fits      9.99   251.43  172.2869   11.984  ok
imageseg2_0.fits    234.10   268.03   179.589    9.542  ok

Analytic psf function(s) (gauss): 	Analytic psf function(s): gauss
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (11.): 	New psf radius: 11. scale units 11. pixels
Fitting radius in scale units (3.): 	New fitting radius: 3. scale units 3. pixels
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Computing PSF for image: /home/mj1e16/Simages/irafImages/imageseg2_0.fits
7 stars read from /home/mj1e16/iraf/outputs/63_2_0daofindone

Star 1 has been added to the PSF star list
	X:  212.37 Y:   18.87  Mag:  -0.951  Dmin: 169.6131  Dmax: 1424

Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts

imageseg2_3.fits     93.03    15.85  181.8104   10.504  ok
imageseg2_3.fits     18.84    17.31  182.0158   10.678  ok
imageseg2_3.fits    107.04    22.31  181.7789   11.451  ok
imageseg2_3.fits     14.90    29.01  186.3354   10.624  ok
imageseg2_3.fits     19.26    41.72  185.3407    9.979  ok
imageseg2_3.fits     74.03    49.62  181.4511   11.543  ok
imageseg2_3.fits     21.45    81.93  183.9216   10.128  ok
imageseg2_3.fits    148.41   126.78  186.4008   10.779  ok
imageseg2_3.fits    128.65   153.75   186.884   11.864  ok
imageseg2_3.fits    117.95   169.40  188.1296   11.738  ok
imageseg2_3.fits     40.99   171.04  191.5244   11.076  ok
imageseg2_3.fits     29.16   173.63  194.3893    9.319  ok
imageseg2_3.fits    115.73   198.74  191.5633   10.055  ok
imageseg2_3.fits    205.91   202.60  185.3368   11.911  ok
imageseg2_3.fits    119.79   221.68  195.7967   11.599  ok
imageseg2_3.fits    158.

Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts

imageseg3_0.fits    238.72    18.74  175.9604   11.913  ok
imageseg3_0.fits    265.01    35.11  179.0949   10.005  ok
imageseg3_0.fits    198.87    37.85   171.807   10.285  ok
imageseg3_0.fits     76.69    51.42  168.5078   11.578  ok
imageseg3_0.fits    258.37    63.93  174.3684   11.221  ok
imageseg3_0.fits    167.60    90.80   167.631   11.039  ok
imageseg3_0.fits     66.11   101.36   170.408   12.027  ok
imageseg3_0.fits     20.47   108.14  170.5113   11.342  ok
imageseg3_0.fits     83.91   124.07  172.9645    9.658  ok
imageseg3_0.fits    202.51   132.71  169.8126   11.501  ok
imageseg3_0.fits    205.13   150.31  170.4639   11.878  ok
imageseg3_0.fits    144.15   151.08  170.2823   10.767  ok
imageseg3_0.fits    143.14   157.16  169.7095   10.225  ok
imageseg3_0.fits    195.55   170.93  168.1967   11.638  ok
imageseg3_0.fits    245.54   198.79  175.0537    9.798  ok

Analytic psf function(s

	X:  233.24 Y:  145.28  Mag:  -1.361  Dmin: 178.9618  Dmax: 163044.4
Star 10 has been added to the PSF star list
	X:   93.17 Y:  170.29  Mag:  -0.148  Dmin: 68.63898  Dmax: 148003.2
Star 11 has been added to the PSF star list
	X:   93.67 Y:  182.57  Mag:  -1.178  Dmin: 68.63898  Dmax: 176648.1

11 PSF stars read from /home/mj1e16/iraf/outputs/63_3_1photone


Fitting function gauss    norm scatter: 0.2851844

Analytic PSF fit 
    Function: gauss  X: 133.5  Y: 120.5  Height: 309834.6  Psfmag: 10.005
    Par1: 0.8420281  Par2: 2.081951

Computed 1 lookup table(s)

Writing PSF image /home/mj1e16/iraf/outputs/63_3_1psfone.fits
Writing output PSF star list /home/mj1e16/iraf/outputs/63_3_1optsone
Writing output PSF star group file /home/mj1e16/iraf/outputs/63_3_1groupfileone
find me
/home/mj1e16/Simages/irafImages/testsimage_3_1_-7.fits
/home/mj1e16/Simages/diff_79_1-7_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New F

In [40]:
for mag in range(-7,0):
    stitchStarList(4,mag,starDir='/home/mj1e16/starlists/44/1/')

In [17]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'global color=lightgreen\nimage\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},5)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase + str(tables)+'.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
        print(fileName)


In [15]:
data = ascii.read('/home/mj1e16/keplerPhotometry/starlistFull.dat')

In [16]:
makeDS9RegFile([data],'/home/mj1e16/Simages/teststarlist','iraf')

/home/mj1e16/Simages/teststarlist0.reg
['44_2_0photone', 'logfile_63_x2_y7', '44photone', 'logfile_9_x8_y13', 'logfile_63_x13_y6', 'starfield_0_0.dat', 'logfile_44_x0_y10', 'logfile_9_x9_y3', '44_3_3photone', 'logfile_44_x13_y2', 'mkObjects.ipynb', 'logfile_44_x13_y12', 'logfile_63_x1_y10', 'logfile_63_x0_y10', 'test19.fits', 'logfile_44_x8_y4', 'login.cl', 'logfile_44_x6_y7', '44_0_4groupfileone', 'logfile_9_x13_y11', '44_0_0photone', 'logfile_9_x9_y2', 'removeStarfield.sh', 'logfile_63_x7_y9', 'logfile_9_x5_y7', 'logfile_63_x13_y13', 'logfile_63_x9_y12', 'test4.fits', 'logfile_63_x6_y5', 'logfile_9_x12_y1', 'imaginary1000.fits', 'logfile_63_x4_y3', 'logfile_9_x12_y2', 'test6.fits', '44_4_1daofindone', 'logfile_44_x6_y9', 'logfile_9_x12_y9', 'logfile_9_x14_y7', 'starfield_1_3.dat', 'logfile_9_x11_y9', 'aaa.sh', 'logfile_44_x8_y9', 'logfile_9_x7_y11', 'logfile_44_x14_y8', 'logfile_63_x7_y2', 'logfile_63_x2_y0', 'logfile_63_x6_y12', 'logfile_9_x0_y9', 'logfile_9_x8_y12', 'logfile_44_x6_

In [2]:
for x in range(-7,0):
    print(x)

-7
-6
-5
-4
-3
-2
-1
